In [ ]:
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow import keras
import os
from os import path, listdir

# Define the function to load the dataset
def load_image_dataset(file_path):
    all_image_dirs = [os.path.join(file_path, f) for f in os.listdir(file_path) if not os.path.isdir(os.path.join(file_path, f))]
    all_image_labels = []
    for f in all_image_dirs:
        if "cat" in f:
            all_image_labels.append(0)
        else:
            all_image_labels.append(1)
    return all_image_dirs, all_image_labels

# Load dataset
train_path = r"C:/Users/LENOVO/Desktop/mlflowshit/data/train"
all_image_dirs, all_image_labels = load_image_dataset(train_path)

# Set up MLflow experiment
mlflow.set_experiment("dog-cat-classification-optimizer-comparison")

# Split the data into training and testing sets
num_train_image = int(len(all_image_labels) * 0.8)
train_image_dirs, train_label = all_image_dirs[:num_train_image], all_image_labels[:num_train_image]
test_image_dirs, test_label = all_image_dirs[num_train_image:], all_image_labels[num_train_image:]

# Create TensorFlow datasets
train_path_label = tf.data.Dataset.from_tensor_slices((train_image_dirs, train_label))
test_path_label = tf.data.Dataset.from_tensor_slices((test_image_dirs, test_label))

# Define the image preprocessing function
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0
    image = 2 * image - 1
    return image

# Prepare the training and testing datasets
train_image_label_ds = train_path_label.map(lambda x, y: (load_and_preprocess_image(x), y))
test_image_label_ds = test_path_label.map(lambda x, y: (load_and_preprocess_image(x), y)).batch(1)

# Define the CNN model
def create_model(optimizer):
    mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
    mobile_net.trainable = False
    cnn_model = keras.models.Sequential([
        mobile_net,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(2, activation="softmax")
    ])
    cnn_model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
    return cnn_model

# Define optimizer variations
optimizers = [tf.keras.optimizers.Adam(), tf.keras.optimizers.SGD(), tf.keras.optimizers.RMSprop()]
EPOCHS = 2
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
steps_per_epoch = len(train_image_dirs) // BATCH_SIZE

# Train models with different optimizers and log results
for optimizer in optimizers:
    with mlflow.start_run():
        mlflow.log_param("optimizer", optimizer.__class__.__name__)
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        
        # Create and fit the model
        cnn_model = create_model(optimizer)
        train_ds = train_image_label_ds.shuffle(buffer_size=len(train_image_dirs)).repeat().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
        history = cnn_model.fit(train_ds, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)

        # Log metrics
        mlflow.log_metric("accuracy", history.history['accuracy'][-1])
        mlflow.log_metric("loss", history.history['loss'][-1])

        # Log the model
        mlflow.keras.log_model(cnn_model, "model")
